In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# # Create an OpenAI model client.
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o-mini",
#     # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
# )

model_client = OpenAIChatCompletionClient(
    model="qwen2.5",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' only when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [21]:
# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="Write a poem on France")
# print(result.messages)

In [18]:
result

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a poem on France', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=362), content="In lands where lavender kisses the breeze,  \nAnd vineyards stretch under sun-dappled trees,  \nWhere castles stand proud on hillsides so grand,  \nFrance weaves a tale with a gracious hand.  \n\nThe Seine flows like silk through Parisian nights,  \nBeneath the soft glow of the city's lights,  \nWith whispers of love in each cobblestone lane,  \nHistory’s secrets in shadows remain.  \n\nMont Saint-Michel rises from tides' gentle reach,  \nA beacon of hope, where the heavens can teach,  \nThe echoes of pilgrims, their dreams in the air,  \nIn the heart of this nation, their stories laid bare.  \n\nFrom the Pyrenees' peaks, where the jagged rocks soar,  \nTo the shores of the Côte d'Azur’s endless lore,  \nEach region a tapestry, woven with care,  \nThe flavors, 

In [19]:
for i in result.messages:
    print(f"{i.source} - {i.content} \n")

user - Write a poem on France 

primary - In lands where lavender kisses the breeze,  
And vineyards stretch under sun-dappled trees,  
Where castles stand proud on hillsides so grand,  
France weaves a tale with a gracious hand.  

The Seine flows like silk through Parisian nights,  
Beneath the soft glow of the city's lights,  
With whispers of love in each cobblestone lane,  
History’s secrets in shadows remain.  

Mont Saint-Michel rises from tides' gentle reach,  
A beacon of hope, where the heavens can teach,  
The echoes of pilgrims, their dreams in the air,  
In the heart of this nation, their stories laid bare.  

From the Pyrenees' peaks, where the jagged rocks soar,  
To the shores of the Côte d'Azur’s endless lore,  
Each region a tapestry, woven with care,  
The flavors, the colors, the songs that we share.  

In markets alive with the scent of fresh bread,  
With laughter and chatter where joy is widespread,  
A sip of rich wine, a bite of delight,  
In the heart of Franc

In [22]:
for i in result.messages:
    print(f"{i.source} - {i.content} \n")

user - Write a poem on France 

primary - Under the Sun's Majestic Hand

Beneath the azure skies so clear and bright,
France, land of light, shines in my sight.
A legacy etched with time’s ancient hand,
The heart of Europe, rich in every land.

From fields where golden wheat does freely sway,
To streets awash in rivers’ shimmering way.
Its heritage is like a tapestry woven,
Of history and tales that linger on.

Roses bloom and lavender doth thrive,
In provinces from Rhône to Brie.
Eiffel Tower lifts its beacon high,
A symbol of strength, of valor, pride and sky.

Mountains rise in rugged splendor grand,
From Alps’ icy crests where snowflakes dance and land.
While the Pyrenees, with their deep purple haze,
Guard secrets and landscapes like no other place.

Paris walks serene on Seine’s soft flow,
Where lovers meet on bridges of love aglow.
Underneath an umbrella of ivy-fumed trees,
In gardens of Versailles, where queens once reigned supreme.

Walls speak of battles fought and tales unto

In [15]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message.content)

Write a short poem about the fall season.
Golden leaves, a crisp embrace,  
Whispering winds in a gentle race.  
Amber hues and muted skies,  
Nature's canvas, where beauty lies.  

Pumpkin scents in the cooling air,  
Harvest bounty, gifts to share.  
A quiet hush, as days grow short,  
Autumn's dance, the year's escort.  

Warmth in wool, fireside dreams,  
A cozy world of cider streams.  
In this season's fleeting grace,  
Time slows down in nature's space.  
Your poem beautifully captures the essence of the fall season with vivid imagery and a cozy atmosphere. Here are a few suggestions that might enhance it further:

1. **Rhythm and Flow:** Ensure each line has a consistent rhythm to create a soothing flow. Consider the syllabic structure of each line, as currently, they vary in length.

2. **Imagery Enhancement:** While the imagery is already appealing, you could consider adding more sensory details. For example, mentioning the sound of rustling leaves or the feeling of a cool br

In [23]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves whisper in hues of gold and red,
As summer's warmth is laid to rest.
Cool breezes dance, trees' arms wide spread,
With tales of forests once becalmed.

Frost kisses leaves, soft and mild,
Glimmering under moonlit shields.
Acorns fall beneath the wise old pines,
Pledging secrets to the silent fields.

Cozy nights are here with starlit skies,
And stories shared by fireside light.
The air is crisp, a breath of sighs,
As autumn's hand upon the earth takes flight.
---------- critic ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=116), content="Leaves whisper in hues of gold and red,\nAs summer's warmth is laid to rest.\nCool breezes dance, trees' arms wide spread,\nWith tales of forests once becalmed.\n\nFrost kisses leaves, soft and mild,\nGlimmering under moonlit shields.\nAcorns fall beneath the wise old pines,\nPledging secrets to the silent fields.\n\nCozy nights are here with starlit skies,\nAnd stories shared by fireside light.\nThe air is crisp, a breath of sighs,\nAs autumn's hand upon the earth takes flight.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=156, completion_tokens=4), content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

In [24]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Endless waves that dance and play,  
Under the sun’s warm golden ray.  
Whispers of secrets from the deep,  
In the ocean's embrace, we dream and keep.


Enter your response:  2 lines only


---------- user_proxy ----------
2 lines only
---------- assistant ----------
Vast and blue, the ocean sighs,  
Beneath the stars, its beauty lies.


Enter your response:  approve


---------- user_proxy ----------
approve
---------- assistant ----------
TERMINATE


Enter your response:  APPROVE


---------- user_proxy ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=39), content="Endless waves that dance and play,  \nUnder the sun’s warm golden ray.  \nWhispers of secrets from the deep,  \nIn the ocean's embrace, we dream and keep.", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='107881f7-8e6d-4d90-a077-f1cbe9b76a98', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='2 lines only', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=99, completion_tokens=22), content='Vast and blue, the ocean sighs,  \nBeneath the stars, its beauty lies.', type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='09ba8cad-4b7c-4836-9ff7-04a83a986b50', cont